In [2]:
from sdf_utils import sap

In [ ]:
eval_test_time_start = time.time()
test_err_sum = 0.
chamfer_dists = []
chamfer_dists_all = []
test_loss_hists = []
mesh_label_names = []
test_latents = []
for test_fname in eval_test_filenames:
    save_name = os.path.basename(sdf_dataset.npyfiles[index]).split(".npz")[0]
    mesh_label_names.append(save_name)
    path = os.path.join(experiment_directory, ws.tb_logs_dir, ws.tb_logs_test_reconstructions, save_name)
    if not os.path.exists(path):
        os.makedirs(path)
    test_fpath = os.path.join(data_source, ws.sdf_samples_subdir, test_fname)
    test_sdf_samples = deep_sdf.data.read_sdf_samples_into_ram(test_fpath)
    test_sdf_samples[0] = test_sdf_samples[0][torch.randperm(test_sdf_samples[0].shape[0])]
    test_sdf_samples[1] = test_sdf_samples[1][torch.randperm(test_sdf_samples[1].shape[0])]

    start = time.time()
    test_loss_hist, test_latent = reconstruct.reconstruct(
        decoder,
        int(eval_test_optimization_steps),
        latent_size,
        test_sdf_samples,
        0.01,  # [emp_mean,emp_var],
        0.1,
        num_samples=8000,
        lr=5e-3,
        l2reg=True,
        return_loss_hist=True
    )
    logging.debug("[Test eval] Total reconstruction time: {}".format(time.time() - start))
    if not np.isnan(test_loss_hist[-1]):
        test_err_sum += test_loss_hist[-1]
    test_loss_hists.append(test_loss_hist)
    test_latents.append(test_latent)

    start = time.time()
    with torch.no_grad():
        test_mesh = mesh.create_mesh(
            decoder, 
            test_latent, 
            N=eval_grid_res, 
            max_batch=int(2 ** 18), 
            filename=os.path.join(path, f"epoch={epoch}"),
            return_trimesh=True,
        )
    logging.debug("[Test eval] Total time to create test mesh: {}".format(time.time() - start))

    if test_mesh is not None:
        gt_mesh_path = f"{torus_path}/{save_name}.obj"
        cd, cd_all = metrics.compute_metric(gt_mesh=gt_mesh_path, gen_mesh=test_mesh, metric="chamfer")
        chamfer_dists.append(cd)
        chamfer_dists_all.append(cd_all)

    del test_sdf_samples, test_mesh

if chamfer_dists:
    logging.debug(f"Test Chamfer distance mean: {sum(chamfer_dists)/len(chamfer_dists)} from {chamfer_dists}.")            
    summary_writer.add_scalar("Mean Chamfer Dist/test", sum(chamfer_dists)/len(chamfer_dists), epoch)
    summary_writer.add_scalar("Loss/test", test_err_sum/len(eval_test_filenames), epoch)
    mlm = torch.mean(torch.norm(torch.cat(test_latents, dim=0), dim=1))
    summary_writer.add_scalar("Mean Latent Magnitude/test", mlm, global_step=epoch)
    fig = plotting.plot_train_stats(loss_hists=test_loss_hists, labels=mesh_label_names)
    summary_writer.add_figure("Loss/test optimization curves", fig, epoch)
    fig, percentiles = plotting.plot_dist_violin(np.concatenate(chamfer_dists_all, axis=0))
    summary_writer.add_figure("CD Percentiles/test dists", fig, global_step=epoch)
    for p in [75, 90, 99]:
        if p in percentiles:
            summary_writer.add_scalar(f"CD Percentiles/test {p}th", percentiles[p], global_step=epoch)
summary_writer.add_scalar("Time/test eval per shape (sec)", (time.time()-eval_test_time_start)/len(eval_test_filenames), epoch)
# End of eval test.

summary_writer.add_scalar("Time/epoch (min)", (time.time()-epoch_time_start)/60, epoch)
summary_writer.flush() 